In [4]:


import tensorflow as tf
import data_providers as data_providers
import numpy as np
import time

batchSize = 50
train_data = data_providers.ACLIMDBDataProvider('train', batch_size=batchSize)
valid_data = data_providers.ACLIMDBDataProvider('valid', batch_size=5000)
# print(train_data.next())
def getTrainBatch():
	# inputs, targets = train_data.next()
    return train_data.next()
def getValBatch():
    return valid_data.next()
def processBatch(batch):
    maxLength = 0
    batchLength = np.zeros([batchSize])
    for i in range(batchSize):
        batchLength[i] = len(batch[i])
        if maxLength < len(batch[i]):
            maxLength = len(batch[i])
    newBatch = np.zeros((batchSize,maxLength))
    for i in range(batchSize):
        newBatch[i] = np.pad(batch[i],(0,maxLength-len(batch[i])),'constant')
    return newBatch, batchLength
            
lstmUnits = 64
numClasses = 2
max_epoch = 20
maxSeqLength = 2505
numDimensions = 300 #Dimensions for each word vector
vocab_size = 93929

tf.reset_default_graph()

input_data = tf.placeholder(tf.int32, [batchSize,None]) #(b,L)
labels = tf.placeholder(tf.float32, [batchSize, numClasses]) #(b,2)
bacthLength = tf.placeholder(tf.int32, [batchSize])

embedding_table = tf.Variable(tf.random_normal([vocab_size, numDimensions]))

data = tf.nn.embedding_lookup(embedding_table, input_data) # data(b,max,d)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)

outputs, final_states = tf.nn.dynamic_rnn(lstmCell, data, bacthLength, dtype=tf.float32) #final_states.h (b,lstmUnits)

weight = tf.Variable(tf.random_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.zeros([numClasses]))

prediction = tf.matmul(final_states.h, weight) + bias

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = labels, logits = prediction))
optimizer = tf.train.AdamOptimizer().minimize(loss)

init = tf.global_variables_initializer()

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merge_summary = tf.summary.merge_all()

with tf.Session() as sess:
    
    train_writer = tf.summary.FileWriter(r"C:\Users\Me\Downloads\mlp project\trainSummary",sess.graph)
    valid_writer = tf.summary.FileWriter(r"C:\Users\Me\Downloads\mlp project\validSummary",sess.graph)
    step = 0
    sess.run(init)
    while step<max_epoch:
        step = step+1
        print("epoch:{}".format(step))
        train_data.new_epoch()
        iterations = 0
        while iterations<train_data.num_batches:
            iterations = iterations + 1
            print("epoch:{}".format(step),end=" ")
            print("iter:{}".format(iterations))
            train_batch = getTrainBatch()
            batch_data, batch_len = processBatch(train_batch[0])
            batch_label = train_batch[1]
            dict_feed = {}
            dict_feed[input_data] = batch_data
            dict_feed[labels] = batch_label
            dict_feed[bacthLength] = batch_len
            sess.run(optimizer, dict_feed)
            print("Acc:{}".format(sess.run(accuracy,dict_feed)))
            if iterations % 100 == 0:
                train_summary = sess.run(merge_summary,dict_feed)
                train_writer.add_summary(train_summary,iterations*step)

                valid_data.new_epoch()
                valid_batch = getValBatch()
                valid_batch_data, valid_batch_len = processBatch(valid_batch[0])
                valid_batch_label = valid_batch[1]
                valid_dict = {}
                valid_dict[input_data] = valid_batch_data
                valid_dict[labels] = valid_batch_label
                valid_dict[bacthLength] = valid_batch_len
        
                
                print("Valid Acc:{}".format(sess.run(accuracy,valid_dict)))
                print("Valid Loss:{}".format(sess.run(loss,valid_dict)))
                
                valid_summary = sess.run(merge_summary, valid_dict)
                valid_writer.add_summary(valid_summary, iterations*step)

epoch:1
epoch:1 iter:1
Acc:0.7400000095367432
epoch:1 iter:2
Acc:0.7599999904632568
epoch:1 iter:3
Acc:0.5400000214576721
epoch:1 iter:4
Acc:0.47999998927116394
epoch:1 iter:5
Acc:0.5799999833106995
epoch:1 iter:6
Acc:0.6200000047683716
epoch:1 iter:7
Acc:0.47999998927116394
epoch:1 iter:8
Acc:0.6000000238418579
epoch:1 iter:9
Acc:0.6399999856948853
epoch:1 iter:10
Acc:0.5199999809265137
epoch:1 iter:11
Acc:0.6200000047683716
epoch:1 iter:12
Acc:0.6000000238418579
epoch:1 iter:13
Acc:0.46000000834465027
epoch:1 iter:14
Acc:0.7200000286102295
epoch:1 iter:15
Acc:0.5
epoch:1 iter:16
Acc:0.5
epoch:1 iter:17
Acc:0.5799999833106995
epoch:1 iter:18
Acc:0.47999998927116394
epoch:1 iter:19
Acc:0.6800000071525574
epoch:1 iter:20
Acc:0.5799999833106995
epoch:1 iter:21
Acc:0.6399999856948853
epoch:1 iter:22
Acc:0.5400000214576721
epoch:1 iter:23
Acc:0.5600000023841858
epoch:1 iter:24
Acc:0.6200000047683716
epoch:1 iter:25
Acc:0.7200000286102295
epoch:1 iter:26
Acc:0.5199999809265137
epoch:1 iter:

ValueError: Cannot feed value of shape (5000, 2) for Tensor 'Placeholder_1:0', which has shape '(50, 2)'